In [1]:
import tensorflow.compat.v2 as tf
import tensorflow_hub as hub

# Import model

In [8]:
# m = hub.KerasLayer('https://tfhub.dev/google/aiy/vision/classifier/food_V1/1')
m = hub.KerasLayer('model')


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
